In [1]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
import keras, math
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Queue import Queue
%matplotlib inline

Using TensorFlow backend.
/gpfs/fs01/user/s745-af92d31e43ed70-980ba6aaa6c3/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_healthy = pickle.load(open('watsoniotp.healthy.pickle', 'rb'))
data_broken = pickle.load(open('watsoniotp.broken.pickle', 'rb'))

In [3]:
data_healthy = data_healthy.reshape(3000,3)
data_broken = data_broken.reshape(3000,3)

In [4]:
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [5]:
data_healthy_scaled = scaleData(data_healthy)
data_broken_scaled = scaleData(data_broken)

In [6]:
timesteps = 10
dim = 3
samples = 300
data_healthy_scaled_reshaped = data_healthy_scaled

In [8]:
# design network

model = Sequential()
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=False))
model.add(Dense(timesteps*dim))
model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.95, decay=5e-4, nesterov=True))

## Now use Keras2DML importer

In [10]:
from systemml.mllearn import Keras2DML
epochs = 50
batch_size = 50
max_iter = int(epochs*math.ceil(samples/batch_size))
model = Keras2DML(spark, model, input_shape=(timesteps, dim, 1), batch_size=batch_size, max_iter=max_iter, test_interval=0, display=10)
model.set(perform_one_hot_encoding=False)
model.set(debug=True) 

Loading the model from /gpfs/fs01/user/s745-af92d31e43ed70-980ba6aaa6c3/notebook/tmp/tmp_flUKW...
SystemML Statistics:
Total execution time:		0.000 sec.
Number of executed Spark inst:	0.




Caffe2DML

In [12]:
def train(data):
    data = data.reshape(samples, timesteps*dim)
    model.fit(data, data) # epochs=50, batch_size=72, validation_data=(data, data), verbose=0, shuffle=False,callbacks=[LossHistory()])

In [13]:
train(data_healthy_scaled)

Time taken to generate training script from Caffe proto: 0.070278461 seconds.
001|debug = TRUE
002|source("nn/layers/lstm.dml") as lstm
003|source("nn/layers/affine.dml") as affine
004|source("nn/layers/l2_loss.dml") as l2_loss
005|source("nn/optim/sgd_nesterov.dml") as sgd_nesterov
006|source("nn/layers/l1_reg.dml") as l1_reg
007|source("nn/layers/l2_reg.dml") as l2_reg
008|X_full_path = ifdef($X, " ")
009|X_full = read(X_full_path)
010|y_full_path = ifdef($y, " ")
011|y_full = read(y_full_path)
012|num_images = nrow(y_full)
013|weights = ifdef($weights, " ")
014|# Initialize the layers and solvers
015|BATCH_SIZE = 50
016|[lstm_1_weight,lstm_1_bias,out0_3,cellState0_3] = lstm::init(BATCH_SIZE,3,50)
017|dcellState0_3 = matrix(0, rows=BATCH_SIZE, cols=50)
018|[dense_1_weight,dense_1_bias] = affine::init(50,30)
019|# Load the weights. Note: keeping the initialization code in case the layer wants to initialize non-weights and non-bias
020|lstm_1_weight = read(weights + "/lstm_1_weight.mtx

Iter:10, training loss:0.7402424226463188, training accuracy:-1
Iter:20, training loss:0.6137985615946043, training accuracy:-1
Iter:30, training loss:0.4943419402874561, training accuracy:-1
Iter:40, training loss:0.3505907102308653, training accuracy:-1
Iter:50, training loss:0.2015497779921026, training accuracy:-1
Iter:60, training loss:0.1807776464306712, training accuracy:-1
Iter:70, training loss:0.15491052753286896, training accuracy:-1
Iter:80, training loss:0.0906057371240747, training accuracy:-1
Iter:90, training loss:0.08331339644063858, training accuracy:-1
Iter:100, training loss:0.06279892274114533, training accuracy:-1
Iter:110, training loss:0.04359085517807979, training accuracy:-1
Iter:120, training loss:0.04592381152112904, training accuracy:-1
Iter:130, training loss:0.04074779926981933, training accuracy:-1
Iter:140, training loss:0.032874132064213066, training accuracy:-1
Iter:150, training loss:0.0339012386421331, training accuracy:-1
Iter:160, training loss:0.